In [6]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

# Constants
VOCAB_SIZE = 100       # Adjust if needed
MAX_SEQ_LENGTH = 10    # Maximum length of caption sequences
IMAGE_FEATURE_SIZE = 2048  # ResNet50 outputs 2048 features
HIDDEN_SIZE = 64
BATCH_SIZE = 2
NUM_EPOCHS = 5

# Vocabulary for tokenizing captions
word_to_index = {}
index_to_word = {}

# Define a simple model
def create_simple_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(IMAGE_FEATURE_SIZE,)),
        tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
        tf.keras.layers.RepeatVector(MAX_SEQ_LENGTH),
        tf.keras.layers.LSTM(HIDDEN_SIZE, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(VOCAB_SIZE, activation='softmax'))
    ])
    return model

# Load data, preprocess captions, and extract 2048-dimensional features using ResNet50
def load_data(image_dir, captions_file):
    global word_to_index, index_to_word
    
    # Initialize ResNet50 model for feature extraction
    feature_extractor = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    
    all_captions = []
    images = []
    captions = []

    # Read captions file and build vocabulary
    with open(captions_file, 'r') as file:
        for line in file:
            image_name, caption_text = line.strip().split(', ', 1)
            all_captions.append(caption_text.split())
            
    # Build vocabulary from captions
    unique_words = set(word for caption in all_captions for word in caption)
    word_to_index = {word: i+1 for i, word in enumerate(unique_words)}
    word_to_index["<PAD>"] = 0  # Padding token
    word_to_index["<END>"] = len(word_to_index) + 1  # End token
    index_to_word = {i: word for word, i in word_to_index.items()}
    
    # Convert captions to integer tokens and extract features
    with open(captions_file, 'r') as file:
        for line in file:
            image_name, caption_text = line.strip().split(', ', 1)
            image_path = image_name
            print(f"Trying to open image: {image_path}")

            if os.path.exists(image_path):
                try:
                    # Load and preprocess image
                    image = Image.open(image_path).resize((224, 224))  # Resize to ResNet50 input size
                    image = np.array(image)
                    image = preprocess_input(image)  # Use ResNet preprocessing
                    image = np.expand_dims(image, axis=0)

                    # Extract features using ResNet50
                    features = feature_extractor.predict(image)
                    images.append(features.flatten())

                    # Convert caption text to token IDs
                    caption_ids = [word_to_index.get(word, 0) for word in caption_text.split()]
                    caption_ids.append(word_to_index["<END>"])  # Add end token
                    # Pad or truncate caption to MAX_SEQ_LENGTH
                    if len(caption_ids) < MAX_SEQ_LENGTH:
                        caption_ids += [0] * (MAX_SEQ_LENGTH - len(caption_ids))
                    else:
                        caption_ids = caption_ids[:MAX_SEQ_LENGTH]
                    captions.append(caption_ids)

                except Exception as e:
                    print(f"Error loading image {image_path}: {e}")
            else:
                print(f"Image not found: {image_path}")

    # Convert lists to numpy arrays
    images = np.array(images)
    captions = np.array(captions)
    return images, captions

# Training function
def train_captioning_model(image_dir, captions_file):
    # Create model
    model = create_simple_model()
    
    # Compile model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Load data
    images, captions = load_data(image_dir, captions_file)
    
    # Train model
    print("Starting training...")
    model.fit(images, captions, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_split=0.5)
    
    return model

# Function to generate a simple caption for an image
def generate_simple_caption(model, image, max_length=10):
    image = tf.expand_dims(image, 0)
    prediction = model.predict(image)
    caption = []
    for i in range(max_length):
        word_id = np.argmax(prediction[0, i])
        word = index_to_word.get(word_id, "<UNK>")  # "<UNK>" for unknown words
        if word == "<END>":
            break
        caption.append(word)
    return ' '.join(caption)

# Main execution
if __name__ == "__main__":
    image_dir = 'image_data'          # Adjust this path to your image directory
    captions_file = 'captions.txt'    # Adjust this path to your captions file
    model = train_captioning_model(image_dir, captions_file)
    
    # Generate a sample caption
    print("\nGenerating sample caption...")
    random_image = tf.random.normal((IMAGE_FEATURE_SIZE,))
    caption = generate_simple_caption(model, random_image)
    print(f"Generated caption: {caption}")


Trying to open image: 1.png
1/1 [==============================] - 1s 928ms/step
Trying to open image: 2.png
1/1 [==============================] - 0s 98ms/step
Trying to open image: 3.png
1/1 [==============================] - 0s 110ms/step
Trying to open image: 4.png
1/1 [==============================] - 0s 107ms/step
Starting training...
Epoch 1/5
1/1 [==============================] - 3s 3s/step - loss: 4.6172 - accuracy: 0.0000e+00 - val_loss: 4.5162 - val_accuracy: 0.0500
Epoch 2/5
1/1 [==============================] - 0s 52ms/step - loss: 4.2461 - accuracy: 0.2000 - val_loss: 4.3720 - val_accuracy: 0.1500
Epoch 3/5
1/1 [==============================] - 0s 40ms/step - loss: 4.0319 - accuracy: 0.3500 - val_loss: 4.2763 - val_accuracy: 0.1500
Epoch 4/5
1/1 [==============================] - 0s 37ms/step - loss: 3.8844 - accuracy: 0.3500 - val_loss: 4.2291 - val_accuracy: 0.1500
Epoch 5/5
1/1 [==============================] - 0s 39ms/step - loss: 3.7548 - accuracy: 0.3500 - val_